# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id') # 정답을 가져옵니다.

In [3]:
# 결측치를 처리하기 전에,
# 결측 여부가 failure를 예측하는데, 유용할 만하다고 도출된
# measurement_3, measurement_5의 결측 여부만 남깁니다.
df_train[['na_1', 'na_2']] = df_train[['measurement_3', 'measurement_5']].isna()
df_test[['na_1', 'na_2']] = df_test[['measurement_3', 'measurement_5']].isna()

In [4]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [5]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [6]:
from sklearn.experimental import enable_iterative_imputer# 구문을 사용하여 실험 단계인 모듈을 활성화하고, 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
# train에 등장하지 않은 수준이 있습니다, test를 포함하여 결측처리 모델을 만듭니다.
s_imp = pd.concat([
        df_train[X_imp + ['product_code']],
        df_test[X_imp + ['product_code']]
], axis=0).groupby('product_code')\
.apply(
    lambda x: IterativeImputer(estimator=LinearRegression(),random_state=123).fit(x[X_imp])
)
# train에 적용합니다.
df_train[X_imp] = df_train[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )
# test에 적용합니다.
df_test[X_imp] = df_test[X_imp + ['product_code']]\
            .groupby('product_code')\
            .apply(
                lambda x: pd.DataFrame(s_imp.loc[x.name].transform(x[X_imp]), index=x.index, columns=X_imp)
            )

In [7]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
# 역시 train에 등장하지 않은 수준을 처리하기 위해 합치니다.
df_mean = pd.concat([
            df_train[['product_code'] + X_mean],
            df_test[['product_code'] + X_mean]
        ]).groupby('product_code')[X_mean].agg('mean')

df_train[X_mean] = df_train.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))
df_test[X_mean] = df_test.groupby('product_code')[X_mean]\
            .apply(lambda x: pd.DataFrame(x.fillna(df_mean.loc[x.name]), index=x.index, columns=x.columns))

In [8]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
# loading은 train에서의 평균으로 결측치를 처리합니다.
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

In [9]:
# 공통으로 사용할 만한 요소입니다.
from sklearn.model_selection import StratifiedKFold, GroupKFold, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

gcv = GroupKFold(4)
scv = StratifiedKFold(5, random_state=123, shuffle=True)
X = df_test.columns.tolist()

def eval_model(model, cv):
    return cross_validate(
        model, df_train[X], df_train['failure'], return_train_score=True, cv=cv, 
                   scoring='roc_auc', groups=df_train['product_code']
    )

def choose_model(model):
    model.fit(df_train[X], df_train['failure'])
    prd = model.predict_proba(df_test[X])[:, 1]
    pd.DataFrame({
        'failure': prd
    }, index=df_test.index).to_csv('answer6.csv')
    return prd

In [10]:
# GroupKFold, group을 기준으로 test에는 train 등장하지 않는 수준으로 데이터를 구성합니다.
for train_idx, test_idx in gcv.split(df_train[X], df_train['failure'], groups=df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(),
        df_train.iloc[test_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [11]:
# Baseline 모델입니다: 문제 3에서 전진선택법으로 뽑아낸 속성을 사용합니다.
from sklearn.linear_model import LogisticRegression
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])

clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
result = eval_model(clf_lr, gcv)
result, np.mean(result['test_score'])

({'fit_time': array([0.01317739, 0.01864767, 0.01153851, 0.0156219 ]),
  'score_time': array([0.01406932, 0.00398922, 0.        , 0.        ]),
  'test_score': array([0.58822089, 0.58492694, 0.58894173, 0.59538985]),
  'train_score': array([0.59262299, 0.59350682, 0.59192443, 0.58956962])},
 0.5893698519267131)

In [12]:
# 모델을 선택합니다.
prd = choose_model(clf_lr)

In [13]:
# 채점을 해봅니다.
roc_auc_score(df_ans['failure'], prd)

0.5883988309352517

In [14]:
# Baseline 모델입니다: 문제 4에서 PCA로 뽑아낸 모델을 선택합니다.
from sklearn.linear_model import LogisticRegression
ct = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA()), ['measurement_{}'.format(i) for i in range(18)]),
    ('std', StandardScaler(), ['loading']),
    ('pt', 'passthrough', ['na_1', 'na_2'])
])

clf_lr2 = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
result = eval_model(clf_lr2, gcv)
result, np.mean(result['test_score'])

({'fit_time': array([0.04585481, 0.03124356, 0.04686522, 0.04686117]),
  'score_time': array([0.00698113, 0.01562142, 0.        , 0.        ]),
  'test_score': array([0.58523328, 0.58381749, 0.58603273, 0.59187292]),
  'train_score': array([0.59481235, 0.59481825, 0.5933697 , 0.59100098])},
 0.586739105414297)

In [15]:
# MLPClassifier도 사용해봅니다.
from sklearn.neural_network import MLPClassifier
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading'] + ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['na_1', 'na_2'])
])

clf_mlp = make_pipeline(ct, MLPClassifier(hidden_layer_sizes=[4, 4], alpha=0.001, random_state=123))
result = eval_model(clf_mlp, gcv)
result, np.mean(result['test_score'])

({'fit_time': array([0.83906794, 0.90419626, 1.53091979, 1.2779057 ]),
  'score_time': array([0.        , 0.01559281, 0.01559067, 0.        ]),
  'test_score': array([0.58102148, 0.57867625, 0.57843643, 0.58456933]),
  'train_score': array([0.60183525, 0.60205625, 0.60462231, 0.60053106])},
 0.5806758736037123)

In [16]:
# RandomForestClassifier 모델도 만들어 봅니다.  (좀더 튜닝했습니다.)
# PCA도 넣어 봅니다.
from sklearn.ensemble import RandomForestClassifier
X_rf = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), X_rf),
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'])
])
clf_rf = make_pipeline(ct, RandomForestClassifier(
    n_estimators=150, max_depth=7, min_samples_split= 512, random_state=123, n_jobs=4
))
result = eval_model(clf_rf, gcv)
result,  np.mean(result['test_score'])

({'fit_time': array([1.27712178, 0.78798437, 0.78106689, 0.81226778]),
  'score_time': array([0.12500024, 0.12499928, 0.12500262, 0.12499905]),
  'test_score': array([0.58251565, 0.57783387, 0.58548793, 0.58836168]),
  'train_score': array([0.63213838, 0.63809805, 0.63598794, 0.63209556])},
 0.5835497811343335)

In [17]:
# XGBoost로 해봅니다.
import xgboost as xgb
X_xgb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_xgb)
])
clf_xgb = xgb.XGBClassifier(
    learning_rate=0.01, n_estimators=300, subsample=0.5, colsample_bytree=0.9, max_depth=2, random_state=123
)
clf_xgb = make_pipeline(ct, clf_xgb)
result = eval_model(clf_xgb, gcv)
result,  np.mean(result['test_score'])

({'fit_time': array([2.17214561, 2.21822596, 2.2182281 , 2.25943184]),
  'score_time': array([0.03124332, 0.03124237, 0.03124428, 0.03124332]),
  'test_score': array([0.58536186, 0.58262546, 0.58334564, 0.59225923]),
  'train_score': array([0.60852592, 0.60910272, 0.6080427 , 0.60689529])},
 0.5858980476811373)

In [18]:
# GradientBoosting 모델을 만들어 봅니다.
from sklearn.ensemble import GradientBoostingClassifier
X_gb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])
clf_gb = make_pipeline(ct, GradientBoostingClassifier(
    n_estimators=100, max_depth=2, learning_rate=0.01, random_state=123
))
result = eval_model(clf_gb, gcv)
result,  np.mean(result['test_score'])

({'fit_time': array([0.97549939, 0.98425627, 0.98414063, 0.96854973]),
  'score_time': array([0.02509451, 0.        , 0.01559591, 0.01562047]),
  'test_score': array([0.58394407, 0.58068345, 0.5846679 , 0.59357819]),
  'train_score': array([0.59490185, 0.59741658, 0.59562136, 0.59252278])},
 0.5857184027349147)

In [19]:
from sklearn.ensemble import VotingClassifier
# 모두 앙상블을 해봅니다. roc_auc에서 확률을 활용하므로 voting 을 soft로 합니다.
clf_vt = VotingClassifier(
    [
        ('lr', clf_lr), #  Baseline: 속성 선택
        #('lr_2', clf_lr2), # PCA + 속성 선택 모델 
        ('gb', clf_gb), # GradientBoost
        #('rf', clf_rf), # Random Forest
        #('xgb', clf_xgb), # xgboost
        #('mlp', clf_mlp) # MLP
    ],
    voting='soft'
)
result = eval_model(clf_vt, gcv)
result, np.mean(result['test_score'])

({'fit_time': array([1.00148964, 0.98787117, 1.01535869, 0.98413444]),
  'score_time': array([0.01881623, 0.01562524, 0.01561999, 0.01562476]),
  'test_score': array([0.58930567, 0.58573059, 0.58939867, 0.59653137]),
  'train_score': array([0.59567237, 0.59693881, 0.59580892, 0.59247848])},
 0.5902415739208746)

In [20]:
# 모델을 선택합니다.
prd = choose_model(clf_vt)

In [21]:
# 채점을 해봅니다.
roc_auc_score(df_ans['failure'], prd)

0.5927039118705035